In [1]:
import os

import scanpy as sc
import pandas as pd
import numpy as np

from tqdm import tqdm
from spatial_metric import spatial_gene_autocorr, plot_compare_spatial_autocorr, get_sampeled_adata, run_cluster_plot


/home/wys/miniconda3/envs/python3.10/lib/python3.10/importlib/__init__.py:126: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/home/wys/miniconda3/envs/python3.10/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
resolutions = [0.3, 0.5, 0.8]
# resolutions = [0.1, 0.3, 0.5]
patient_ids = ["P1CRC", "P2CRC", "P5CRC"]
patient_ids = ["P2CRC"]
data_type = "HD"
cell_type_col = "Level1"

sample_size = 30000

raw_data_path = "/home/wys/Sim2Real-ST/REVISE_data_process/raw_data"
svc_data_path = "../REVISE/results/HD"
save_dir = "output/sp_SVC_case"


In [3]:
for patient_id in tqdm(patient_ids, desc="Processing patients"):
            
    save_path = f"{save_dir}/{patient_id}_{data_type}"
    os.makedirs(save_path, exist_ok=True)


    print(f"Processing original data for {patient_id}_{data_type}")
    adata_sp = sc.read(f"{raw_data_path}/{patient_id}_{data_type}.h5ad")
    adata_sp = adata_sp[adata_sp.obs[cell_type_col] != "Unknown"].copy()

    if sample_size is not None:
        adata_sp = get_sampeled_adata(adata_sp, sample_size=sample_size, seed=0)
        print(adata_sp.obs[cell_type_col].value_counts())

    run_cluster_plot(adata_sp, f"{save_path}/original", 
                        resolutions = resolutions, 
                        cell_type_col = "Level1") 
    moranI_sp, gearyC_sp, _, _ = spatial_gene_autocorr(adata_sp, cell_type_col = "Level1", save_dir = f"{save_path}/original")
    

    print(f"Processing SVC data for {patient_id}_{data_type}")
    adata_sp_svc = sc.read(f"{svc_data_path}/{patient_id}/{patient_id}_{data_type}_pot_REVISE.h5ad")
    adata_sp_svc.X = adata_sp_svc.layers["ot_smooth"]
    # adata_sp_svc.X.data = np.round(adata_sp_svc.X.data, decimals=0)
    adata_sp_svc = adata_sp_svc[adata_sp_svc.obs[cell_type_col]!= "Unknown"].copy()
    
    if sample_size is not None:
        adata_sp_svc = get_sampeled_adata(adata_sp_svc, sample_size=sample_size, seed=0)
        print(adata_sp.obs[cell_type_col].value_counts())

    run_cluster_plot(adata_sp_svc, f"{save_path}/sp_SVC", 
                        resolutions = resolutions,
                        cell_type_col = "Level1")
    moranI_sp_svc, gearyC_sp_svc, _, _ = spatial_gene_autocorr(adata_sp_svc, cell_type_col = "Level1", 
                                                                save_dir = f"{save_path}/sp_SVC")
    
    plot_compare_spatial_autocorr(moranI_sp_svc, moranI_sp, save_dir = save_path, mode = "moran")
    plot_compare_spatial_autocorr(gearyC_sp_svc, gearyC_sp, save_dir = save_path, mode = "geary")

Processing patients:   0%|          | 0/1 [00:00<?, ?it/s]

Processing original data for P2CRC_HD
Sampling 30000 cells from 545842 total cells.
Level1
Tumor                    12884
Fibroblast                3830
Plasma                    2941
Intestinal Epithelial     2742
SMC                       2304
Mono/Macro                2251
Vascular EC                996
Pericyte                   677
DC                         526
T                          522
Lymphatic EC               151
Mast                        65
Gliacyte                    63
B                           48
Name: count, dtype: int64
Highly variable genes filtering for 16792 genes.
Number of clusters for leiden resolution 0.3: 9
Number of clusters for leiden resolution 0.5: 15
Number of clusters for leiden resolution 0.8: 17
   resolution       ARI       NMI  cluster_num
0         0.3  0.292623  0.529706            9
0         0.5  0.188230  0.490299           15
0         0.8  0.196855  0.503832           17
